# Lab 1

In [18]:
import numpy as np
from itertools import product

def generate_binary_vectors(length):
    """Генерація всіх можливих бінарних векторів заданої довжини."""
    return list(product([0, 1], repeat=length))

def target_function(q, binary_vectors):
    """Цільова функція для мінімізації."""
    return (1/(2**10)) * sum((q - np.sum(binary_vector))**2 for binary_vector in binary_vectors)

def minimize_function(binary_vectors):
    """Мінімізація цільової функції."""
    best_q = None
    best_value = float('inf')

    for q_candidate in range(6):  # q приймає значення від 0 до 5 включно
        current_value = target_function(q_candidate, binary_vectors)
        if current_value < best_value:
            best_value = current_value
            best_q = q_candidate

    return best_q, best_value

if __name__ == "__main__":
    binary_vectors = generate_binary_vectors(5)
    optimal_q, minimal_value = minimize_function(binary_vectors)

    print("Оптимальне значення q:", optimal_q)
    print("Мінімальне значення функції:", minimal_value)
    print("Бінарні згенеровані вектори:", binary_vectors)

Оптимальне значення q: 2
Мінімальне значення функції: 0.046875
Бінарні згенеровані вектори: [(0, 0, 0, 0, 0), (0, 0, 0, 0, 1), (0, 0, 0, 1, 0), (0, 0, 0, 1, 1), (0, 0, 1, 0, 0), (0, 0, 1, 0, 1), (0, 0, 1, 1, 0), (0, 0, 1, 1, 1), (0, 1, 0, 0, 0), (0, 1, 0, 0, 1), (0, 1, 0, 1, 0), (0, 1, 0, 1, 1), (0, 1, 1, 0, 0), (0, 1, 1, 0, 1), (0, 1, 1, 1, 0), (0, 1, 1, 1, 1), (1, 0, 0, 0, 0), (1, 0, 0, 0, 1), (1, 0, 0, 1, 0), (1, 0, 0, 1, 1), (1, 0, 1, 0, 0), (1, 0, 1, 0, 1), (1, 0, 1, 1, 0), (1, 0, 1, 1, 1), (1, 1, 0, 0, 0), (1, 1, 0, 0, 1), (1, 1, 0, 1, 0), (1, 1, 0, 1, 1), (1, 1, 1, 0, 0), (1, 1, 1, 0, 1), (1, 1, 1, 1, 0), (1, 1, 1, 1, 1)]


# Lab 3

In [1]:
import numpy as np
from scipy.stats import norm


def generate_data(n, p_k1, a1, a2):
    """Генерація випадкових величин згідно з умовами задачі."""
    k_states = np.random.choice([1, 2], size=n, p=[p_k1, 1 - p_k1])
    means = np.where(k_states == 1, a1, a2)
    data = np.random.normal(loc=means, scale=1)
    return data, k_states

def expectation_maximization(data, max_iterations=10000, tol=0.001):
    """Алгоритм самонавчання."""
    n = len(data)
    p_k1 = 0.5
    p_k2 = 1 - p_k1
    a1 = 1
    a2 = 2

    for iteration in range(max_iterations):
        # Expectation step
        pdf_k1 = norm.pdf(data, loc=a1, scale=1)
        pdf_k2 = norm.pdf(data, loc=a2, scale=1)
        gamma_k1 = p_k1 * pdf_k1 / (p_k1 * pdf_k1 + p_k2 * pdf_k2)
        gamma_k2 = p_k2 * pdf_k2 / (p_k1 * pdf_k1 + p_k2 * pdf_k2)

        # Maximization step
        p_k1_new = np.mean(gamma_k1)
        p_k2_new = np.mean(gamma_k2)
        
        f1 = norm.pdf(data, loc=0, scale=1)
        f2 = norm.pdf(data, loc=1, scale=1)
        f3 = norm.pdf(data, loc=2, scale=1)
        f4 = norm.pdf(data, loc=3, scale=1)
        
        
        func1 = np.sum(gamma_k1 *np.log(f1))
        func2 = np.sum(gamma_k1 *np.log(f2))
        if (func1 > func2): 
            a1_new = 0
        else: 
            a1_new = 1
            
        func3 = np.sum(gamma_k2 *np.log(f3))
        func4 = np.sum(gamma_k2 *np.log(f4))
        if (func3 > func4): 
            a2_new = 2
        else: 
            a2_new = 3

        # Check convergence
        if np.abs(p_k1_new - p_k1) < tol and  a1_new - a1 == 0 and \
           np.abs(p_k2_new - p_k2) < tol and a2_new - a2 == 0:
            break

        # Update parameters
        p_k1, p_k2, a1, a2 = p_k1_new, p_k2_new, a1_new, a2_new

    return p_k1, p_k2, a1, a2

if __name__ == "__main__":
    np.random.seed(42)  # для відтворюваності результатів
    n = 100
    p_k1_true = 1/3
    a1_true = 0
    a2_true = 3

    data, true_k_states = generate_data(n, p_k1_true, a1_true, a2_true)

    # Ініціалізація параметрів 
    p_k1_init = 0.5
    a1_init = 1
    a2_init = 2

    # Застосування алгоритму самонавчання
    p_k1_hat, p_k2_hat, a1_hat, a2_hat = expectation_maximization(data)

    print("Справжні значення:")
    print("P(k=1) =", p_k1_true, "a1 =", a1_true)
    print("P(k=2) =", 1 - p_k1_true, "a2 =", a2_true)

    print("\nОцінені значення:")
    print("P(k=1) =", p_k1_hat, "a1 =", a1_hat)
    print("P(k=2) =", p_k2_hat, "a2 =", a2_hat)

Справжні значення:
P(k=1) = 0.3333333333333333 a1 = 0
P(k=2) = 0.6666666666666667 a2 = 3

Оцінені значення:
P(k=1) = 0.5341101250146805 a1 = 1
P(k=2) = 0.4658898749853195 a2 = 3


# Lab 2

In [25]:
import numpy as np
from scipy.optimize import minimize

# Генерація векторів
N = 50
vectors = np.zeros((N, 3))
vectors[:, :2] = np.random.exponential(scale=1, size=(N, 2))
vectors[:, 2] = np.random.normal(size=N)

# Реалізація алгоритму Козинця
alpha = vectors[0]
while True:
    dot_products = np.dot(vectors, alpha)
    j = np.argmin(dot_products)
    if dot_products[j] > 0:
        break
    result = minimize(lambda k: np.linalg.norm((1 - k) * alpha + k * vectors[j]), 0)  # Мінімізація виразу
    k = result.x[0]
    alpha = (1 - k) * alpha + k * vectors[j]

print('Розділяючий вектор : ',alpha)
print('Згенеровані вектори : ',vectors)


Розділяючий вектор :  [0.51126151 0.99109083 0.12330675]
Згенеровані вектори :  [[ 7.16862812e-01  2.88052027e+00 -1.16880374e+00]
 [ 4.58042439e-01  1.07833317e+00 -1.61493938e+00]
 [ 1.70354400e+00  1.64346649e+00  5.22571143e-01]
 [ 1.05789445e+00  2.01249247e+00  7.73221256e-01]
 [ 7.92018084e-02  2.31224071e-01  7.13535807e-01]
 [ 1.85233392e+00  2.70596690e-02 -1.41941251e-01]
 [ 5.47287178e-01  7.31454698e-01 -8.33771461e-01]
 [ 4.16116968e-01  1.53838951e+00  3.65296195e-01]
 [ 9.15464735e-01  6.03188426e-01 -5.08459860e-01]
 [ 6.41407396e-02  3.39011366e-01  1.14756534e+00]
 [ 2.15348269e+00  1.45394304e+00 -1.17745247e+00]
 [ 1.74402376e-02  1.81382156e-01 -1.20122717e+00]
 [ 8.94352130e-01  2.39738625e-01  9.78086135e-01]
 [ 8.16249850e-01  1.19799849e+00 -2.80429594e+00]
 [ 4.86891070e-01  1.79340228e+00  1.61661567e-01]
 [ 6.83306246e-01  3.77832119e-01 -1.75916593e-01]
 [ 6.70219060e-01  7.85116173e-02  2.75565081e-01]
 [ 3.37871909e+00  1.60216781e+00  1.26884431e+00]
 [